In [ ]:
from copy import deepcopy

import arrow

from plastering.inferencers.zodiac import ZodiacInterface
from plastering.inferencers.building_adapter import BuildingAdapterInterface
from plastering.metadata_interface import *
from plastering.workflow import Workflow

In [ ]:
target_building = 'ebu3b'
source_buildings = ['ap_m','ebu3b']
target_srcids = LabeledMetadata.objects(building=target_building).distinct('srcid')

In [ ]:
base_config = {
    'target_building': target_building,
    'target_srcids': target_srcids,
    'source_buildings': source_buildings,
}
ba_config = deepcopy(base_config)
ba_config['config'] = {
    'target_time_ranges': [
        (arrow.get(2016, 2, 1), arrow.get(2016, 2, 6))
    ],
    'source_time_range': (arrow.get(2017, 1, 20), arrow.get(2017, 2, 6)),
    #TODO: Clean up this inconsistency in the interface
    'threshold': 0.99999999
}
zodiac_config = deepcopy(base_config)

inferencer_classes = {
    'ba': BuildingAdapterInterface,
    'zodiac': ZodiacInterface,
}
inferencer_graph = { #Currently only Tree is implemented.
    'ba': (ba_config, {
        'zodiac': (zodiac_config, {})
    })
}

In [ ]:
workflow = Workflow(target_srcids,
                    target_building,
                    inferencer_classes,
                    inferencer_graph,
                   )

In [ ]:
workflow.learn_auto() # The wrapper of continuous active learning.
# Print the history of the inferences.
print(workflow.history)